In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from arcgis.geometry import Geometry, SpatialReference, Point, Polygon
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
import datetime as dt
from datetime import datetime
import uuid,json,arcpy
import os

In [ ]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
find_storymaps = gis.content.search("title: Community Storymap NOT Census", item_type="Web Mapping Application", max_items=480, outside_org=False)


In [ ]:
#Set up a log file to write errors to
today = [dt.date.today()]
log_path = r"T:\CDO_To_Esri\Storymaps\_Archive\Logfiles\Communities\Update Municipal Officials"
log_file = os.path.join(log_path, "Update Muncipal Officials_" + str(today[0]) + str(".txt"))
out = open(log_file,'w')

for f in find_storymaps:
    try:
        print(f.title, f.id)
        if f.title != 'Alaska DCRA Community Storymaps' and not f.title.startswith('Communit') and 'Community' in str(f.title) and not 'Borough Storymap' in str(f.title) and not 'Census' in str(f.title):
            csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
            sections = csmap.properties["values"]["story"]["sections"]
            panel_elect = sections[6]
            community = f.title.replace(" Community Storymap", "").replace(" (Unincorporated Community)", "").replace("'","''")
            if community.startswith("City & Borough of "):
                community = community.replace("City & Borough of ", "")
            if community.startswith("Municipality of "):
                community = community.replace("Municipality of ","")
            print(community)
            query = "CommunityName = '" + str(community.replace("'","''")) + "'"


    ### MUNICIPAL OFFICIALS PANEL EXTENTS AND FILTERS ###                      
            #City Council
            cc_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Govt_Related/Govt_Elected_Officials/MapServer/0')
            council_query = cc_base.query(where=query, out_fields="*").df
            if str(council_query) != 'Empty DataFrame\nColumns: []\nIndex: []':
                council_tbl = ['<table border="1"><th width="40%">City Council Member</th><th width="30%">Term End Date</th><tbody>']
                for m in council_query.OfficialName:
                    m_query = cc_base.query(where="OfficialName = '" + str(m.replace("'","''")) + "'", out_fields="*").df
                    member = m_query.OfficialName[0]
                    eot = str(m_query.TermEnds[0])
                    if eot == 'None':
                        eot = 'No data'
                    else:
                        eot = str(datetime.utcfromtimestamp(int(eot[0:10])))
                        end_of_term = datetime.strftime(datetime.strptime(eot, "%Y-%m-%d %H:%M:%S"), '%m/%d/%Y')
                    council_tbl.append('<tr><td>' + str(member) + '</td><td>' + str(end_of_term) + '</td></tr>')
                council_tbl.append('</tbody></table>')
                embed_cc_tbl = ' '.join(council_tbl)
            else:
                embed_cc_tbl = '<i>No data: either this community is unincorporated, or city council information is currently unavailable.</i>'

            #Borough Assembly
            #Find regions for regional web map filters
            region_lyr = FeatureLayer('https://maps.commerce.alaska.gov/server/rest/services/Community_Related/Community_Regions_Overview/MapServer/0')
            region_query = region_lyr.query(where=query,out_fields="*")
            borough = region_query.df.Borough_Census_Area[0]
            print(borough)
            boro_base = gis.content.get("3631c17312c34b6f8dfb9ccfc2ae4b5c").layers[0]
            boro_query = boro_base.query(where="EntityName = '" + str(borough) + "'", out_fields="*").df
            if str(boro_query) == 'Empty DataFrame\nColumns: []\nIndex: []': 
                embed_boro_tbl = "<i> This community is not part of a borough. It does not have a borough assembly.</i>"
            else:
                boro_officials = boro_query.OfficialName
                boro_tbl =['<table border=1><th width="40%"><b>Borough Assembly Member</b></th><th width="50%"><b>Term End Date</b></th><tbody>']
                for b in boro_officials:
                    boro_data = boro_base.query(where="OfficialName = '" + str(b) + "'", out_fields = "*")
                    boro_dt = str(boro_data.df.TermEnds[0])
                    # Time comes in as a UNIX timestamp, convert to a simple mm/dd/yyyy format
                    if boro_dt == 'None':
                        boro_term_dt = 'no data'
                    else:
                        boro_term_unix = str(datetime.utcfromtimestamp(int(boro_dt[0:10])))
                        boro_term_dt = datetime.strftime(datetime.strptime(boro_term_unix, "%Y-%m-%d %H:%M:%S"), '%m/%d/%Y')
                    boro_tbl.append("<tr><td>" + str(b) + "</td><td>" + str(boro_term_dt) + "</td></tr>")
                boro_tbl.append("</tbody></table>")
                embed_boro_tbl = ' '.join(boro_tbl)

            #Mayor
            mayor_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Govt_Related/Govt_Elected_Officials/MapServer/8')
            mayor_query = mayor_base.query(where=query, out_fields="*").df
            if str(mayor_query) != 'Empty DataFrame\nColumns: []\nIndex: []':
                mayor = mayor_query.OfficialName[0]
                mayor_end = str(mayor_base.query(where=query, out_fields="*").df.TermEnds[0])
                if mayor_end == 'None':
                    mayor_end_dt = 'no data'
                else:
                    mayor_term = str(datetime.utcfromtimestamp(int(mayor_end[0:10])))
                    mayor_end_dt = datetime.strftime(datetime.strptime(mayor_term, "%Y-%m-%d %H:%M:%S"), '%m/%d/%Y')
            else:
                mayor = '<i>This community does not have a mayor, either because it is unincorporated, or that data is unavailable.</i>'
                mayor_end_dt = 'N/A'
            #City planning commission
            cp_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Govt_Related/Govt_Elected_Officials/MapServer/10')
            cp_query = cp_base.query(where=query, out_fields="*").df
            if str(cp_query) != 'Empty DataFrame\nColumns: []\nIndex: []':
                cp_tbl = ['<table border="1"><th width="40%">City Planning Commission Member</th><th width="30%">Term End Date</th><tbody>']
                for m in cp_query.OfficialName:
                    if 'City Council' in str(m):
                        member = 'City Council serves as the City Planning Commission in ' + str(community) + "."
                        end_of_term = ' See city council table.'
                    else:
                        m_query = cp_base.query(where="OfficialName = '" + str(m.replace("'","''")) + "'", out_fields="*").df
                        member = m_query.OfficialName[0]
                        eot = str(m_query.TermEnds[0])
                        if eot == 'None':
                            end_of_term = 'No data'
                        else:
                            eot = str(datetime.utcfromtimestamp(int(eot[0:10])))
                            end_of_term = datetime.strftime(datetime.strptime(eot, "%Y-%m-%d %H:%M:%S"), '%m/%d/%Y')
                    cp_tbl.append('<tr><td>' + str(member) + '</td><td>' + str(end_of_term) + '</td></tr>')
                cp_tbl.append('</tbody></table>')
                embed_cp_tbl = ' '.join(cp_tbl)
            else:
                embed_cp_tbl = '<i>No data: this community is either unincorporated, or city planning commission info is currently unavailable. </i>'  

        #House and senate reps
        #Get house and senate districts for the community
            #try:
            election_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Govt_Related/Govt_Elected_Officials/MapServer/4')
            elect_query = election_base.query(where="CommunityName = '" + str(community.replace("'","''")) + "'",out_fields="*").df
            print("Election feature queried.")
            if str(elect_query) != 'Empty DataFrame\nColumns: []\nIndex: []':
                elect_reg = elect_query.ElectionRegion[0]
                district_lst = ['<table border="1"><th width="15%">House District</th><th width="15%">Senate District</th><th width="20%">House Representative</th><th width="20%">Senator</th><th width="30%">District Description</th><tbody>'] 
                for h in elect_query.HouseDistrict:
                    h_item = election_base.query(where="HouseDistrict = " + str(h), out_fields="*").df
                    senate = h_item.SenateDistrict[0]
                    desc = h_item.DistrictDescription[0]
                    sen = h_item.Senator[0]
                    hous = h_item.HouseRepresentative[0]
                    district_lst.append('<tr><td>' + str(h) + '</td><td>' + str(senate) + '</td><td>' + str(hous) + '</td><td>' + str(sen) + '</td><td>' + str(desc) + '</td></tr>')
                district_lst.append('</tbody></table>')
                embed_house_senate = ' '.join(district_lst)
            else:
                embed_house_senate = '<i> House and senate data unavailable </i>'


            #Local Option
            lo_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Govt_Related/Govt_Municipal_Powers/MapServer/3')
            lo_query = lo_base.query(where="CommunityName LIKE '%" + str(community.replace("'","''")) + "%'").df
            if str(lo_query) != 'Empty DataFrame\nColumns: []\nIndex: []':
                alcohol = lo_query.LocalAlcoholOptionRestrictions[0]
                marijuana = lo_query.RestrictionsOnMarijuana[0]
            else:
                alcohol = '<i>Data on alcohol restrictions unavailable for this community.</i>'
                marijuana = '<i>Data on marijuana restrictions unavailable for this community.</i>'
            #Print officials tables to the panel text.
            panel_elect["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue"data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p><p>&nbsp;</p><p>' + str(community) + ' is included in election region ' + str(elect_reg) + '.</p><p>&nbsp;</p><p>' + str(embed_house_senate) + '</p><p>&nbsp;</p><p><table border="1"><th width="40%">Mayor</th><th width="50%">Term End Date</th><tbody><tr><td>' + str(mayor) + '</td><td>' + str(mayor_end_dt) + '</td></tr></tbody></table></p><p>&nbsp;</p><p><b><u>City Council:</u></b></p><div><p>' + str(embed_cc_tbl) + '</p></div><p>&nbsp;</p><p><b><u>City Planning Commission:</u></b></p><div><p>' + str(embed_cp_tbl) + '</p></div><p>&nbsp;</p><p><b><u>Borough Assembly:</u></b></p><div><p>' + str(embed_boro_tbl) + '</p></div><p>&nbsp;</p><p><b>Marijuana Restrictions: </b>' + str(marijuana) + '</p><p><b>Alcohol Restrictions: </b>' + str(alcohol) + '</p>'

            csmap.save()
            print("Municipal officals data posted to panel.")
    except Exception as e:
        print("------- ISSUE WITH " + str(f.title) + ".-------- " + str(e))
        out.write("Municipal Officals panel for " + str(f.title) +  " not properly updated. See storymap for more info.")
        out.write('\n\n')
        print("Community written to error logfile.")
        continue

print('Storymaps municipal offical update complete.')
out.close()